### 读写文件


In [1]:
import torch
from torch import nn
from torch.nn import functional as F 

#### 加载和保存张量
对于单个张量。直接调用load和save函数分别读写，这两个函数需要提供一个名称，save要求保存的变量作为输出

In [2]:
x=torch.arange(4)
torch.save(x, "x-five")

将文件中存储的数据读回内存

In [3]:
x2=torch.load("x-five")
x2

tensor([0, 1, 2, 3])

可以存储一个张量列表，将它读回内存

In [4]:
y=torch.zeros(4)
torch.save([x, y], "x-five")
x2, y2=torch.load("x-five")
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以写入或取从字符串映射到张量的字典，当我们要读取或写入模型中的权重时，这很方便。

In [5]:
mydict={'x' :x, 'y': y}
torch.save(mydict, 'mydict')
mydict2= torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

#### 加载和保存模型参数

In [6]:


class MLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.hidden=nn.Linear(20, 256)
        self.output =nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net=MLP()
X=torch.randn(size=(2, 20))
Y=net(X)

#将模型参数存储在一个mlp.params文件中
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们实例化了原始多层感知机模型的一个备份。 这里我们不需要随机初始化模型参数，而是直接读取文件中存储的参数

In [7]:
clone=MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [8]:
Y_clone=clone(X)
Y_clone==Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

#### GPU
在PyTorch中，CPU和GPU可以用torch.device('cpu') 和torch.device('cuda')表示。 应该注意的是，cpu设备意味着所有物理CPU和内存， 这意味着PyTorch的计算将尝试使用所有CPU核心。 然而，gpu设备只代表一个卡和相应的显存。 如果有多个GPU，我们使用torch.device(f'cuda:{i}') 来表示第块GPU（从0开始）。 另外，cuda:0和cuda是等价的。

In [12]:
import torch
from torch import nn

torch.device('gpu'), torch.device('cuda'), torch.device('cuda: 1')

RuntimeError: Expected one of cpu, cuda, xpu, mkldnn, opengl, opencl, ideep, hip, ve, ort, mlc, xla, lazy, vulkan, meta, hpu device type at start of device string: gpu

In [ ]:
torch.cuda.device_count()

1

In [13]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    return npx.gpu(i) if npx.num_gpus() >= i + 1 else npx.cpu()

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu()]"""
    devices = [npx.gpu(i) for i in range(npx.num_gpus())]
    return devices if devices else [npx.cpu()]

try_gpu(), try_gpu(10), try_all_gpus()

NameError: name 'npx' is not defined

In [14]:
#存储在GPU
import numpy as np
X=np.ones((2, 3), ctx=try_gpu())

NameError: name 'npx' is not defined